list google bucket url map?


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials = path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

function listUrlMaps(project, urlMap) {
    var params = {};
    if(urlMap) {
        params.filter = `name="${urlMap}"`
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = {
                    hostRules: cur.hostRules,
                    pathMatchers: cur.pathMatchers,
                    fingerprint: cur.fingerprint
//    defaultService: 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/backendServices/web-map-backend-service',
                };
                return obj;
            }, {})
        });
}

function listTargetProxies(project, urlMap, bucketName) {
    var params = {};
    if(urlMap) {
        params.filter = `urlMap="https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}"`
    }
    if(bucketName) {
        // TODO search ssl certs base on hostname instead of cert name?
        params.filter = `sslCertificate="https://www.googleapis.com/compute/v1/projects/${project}/global/sslCertificates/cert-${bucketName}`;
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies`,
            params
        }))
        .then(res => (res.data.items || []).map(p => p.name));
}

function listGlobalForwards(project, proxy, ip) {
    var params = {};
    if(proxy) {
        params['filter'] = `target="https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies/${proxy}"`
    }
    if(ip) {
        params['ip'] = `IPAddress="${ip}"`;
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/forwardingRules`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = {
                    IPAddress: cur.IPAddress,
                    target: cur.target
                }
                return obj;
            }, {})
        });
}

function listBackendBuckets(project, bucketName) {
    var params = {};
    if(bucketName) {
        params.filter = `bucketName="${bucketName}"`;
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = cur.bucketName;
                return obj;
            }, {})
        });
}

module.exports = {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets
};


add a google bucket web map?


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials = path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');
var uuid = require('uuid/v1');
var {
    listBackendBuckets,
    listTargetProxies,
    listGlobalForwards,
    listUrlMaps
} = importer.import('list google bucket url map');

function safeName(name) {
    return name.replace(/[^a-z\-]/ig, '-').substr(0, 50);
}

function insertBackendBucket(project, bucketName) {
    // TODO: check for creation of bucket here?
    
    return listBackendBuckets(project, bucketName)
        .then(buckets => {
            if(Object.keys(buckets).length > 0) {
                console.log(`bucket ${bucketName} already mapped`);
                return Promise.resolve(Object.keys(buckets)[0]);
            }
            return authorizeGoogle(credentials)
                .then(client => client.request({
                    method: 'POST',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets`,
                    data: {
                        name: 'bb-' + uuid().substr(0, 5) + safeName(bucketName),
                        bucketName: bucketName,
                        enableCdn: false,
                        kind: 'compute#backendBucket'
                    }
                }))
                .then(bucket => bucket.name)
        })
}

function insertTargetHttpsProxy(project, urlMap, bucketName) {
    // TODO: get SSL cert first because it's require by proxy
    
    // check for a proxy with an sslCert matching our project name
    return listTargetProxies(project, urlMap, bucketName)
        .then(proxies => {
            if(proxies.length > 0) {
                console.log(`proxy ${bucketName} already exists`);
                return Promise.resolve(proxies[0]);
            }
            return authorizeGoogle(credentials)
                .then(client => client.request({
                    method: 'POST',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies`,
                    data: {
                        name: 'thp-' + uuid().substr(0, 5) + safeName(bucketName),
                        urlMap: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}`,
                        sslCertificates: [],
//                        sslPolicy: 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/sslPolicies/my-ssl-policy'
                    }
                }))
                .then(proxy => rule.name)
        })
}

function insertGlobalForward(project, ip, urlMap, bucketName) {
    return listGlobalForwards(project, ip)
        // add a global forward if the ip isn't already set up
        .then(rules => {
            if(Object.keys(rules.length) > 0) {
                console.log(`forward ${ip} already exists`);
                return Promise.resolve(Object.keys(buckets)[0]);
            }
            // get proxy first because it's require by forwardRule
            var targetHttpsProxy;
            return insertTargetHttpsProxy(project, urlMap, bucketName)
                .then(proxy => (targetHttpsProxy = proxy))
                .then(() => authorizeGoogle(credentials))
                .then(client => client.request({
                    method: 'POST',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/forwardingRules`,
                    data: {
                        name: 'gfr-' + uuid().substr(0, 5) + safeName(urlMap),
                        IPAddress: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses/${ip},
                        IPProtocol: 'TCP',
                        portRange: '443-443',
                        target: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies/${targetHttpsProxy}`,
                        loadBalancingScheme: 'EXTERNAL',
                        networkTier: 'PREMIUM'
                    }
                }))
                .then(rule => rule.name)
        });
}

function updateUrlMap(project, urlMap, bucketName) {
    return listUrlMaps(project, urlMap)
        // add a global forward if the ip isn't already set up
        .then(maps => {
            if(typeof maps[urlMap] == 'undefined') {
                // TODO: create the map
                throw new Error('Implement url map create');
            }
            if(maps[urlMap].hostRules.filter(hr => hr.hosts.filter(h => h == bucketName).length > 0).length > 0) {
                console.log(`map host rule ${bucketName} already exists`);
                return Promise.resolve(urlMap);
            }
            console.log(`map ${urlMap} already exists, adding host rule`);
            var defaultService;
            var pathMatcherName = 'pm-' + uuid().substr(0, 5) + safeName(bucketName);
            return insertBackendBucket(project, bucketName)
                .then(bn => (defaultService = bn))
                .then(() => authorizeGoogle(credentials))
                .then(client => client.request({
                    method: 'PATCH',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps`,
                    data: {
                        name: safeName(urlMap),
                        fingerprint: maps[urlMap].fingerprint,
                        hostRules: maps[urlMap].hostRules.concat([{
                            hosts: [bucketName],
                            pathMatcher: pathMatcherName
                        }]),
                        pathMatchers: maps[urlMap].pathMatchers.concat([{
                            name: pathMatcherName,
                            defaultService: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets/${defaultService}`,
                            pathRules: ['/', '/*']
                        }]),
                    }
                }))
                .then(map => map.name)
        });
}

module.exports = {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
};


test bucket web map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');
var project = 'spahaha-ea443';
var domain = 'sheet-to-web.sheet-to-web.com';
var urlMap = 'web-map';

describe('adding a subdomain to a load balancer', () => {
    it('should add a bucket backend', () => {
        return insertBackendBucket(project, domain)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a backend');
            })
    })
    
    it('should add a frontend to load balancer', () => {
        // TODO: check for A record in DNS to get ip
        return insertGlobalForward(project, ip, urlMap, domain)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a forward');
            })
    })
    
    it('should update url map on load balancer', () => {
        return updateUrlMap(project, urlMap, domain)
            .then(map => {
                assert(map.length > 0, 'should have updated the map');
            })
    })
})


how to check dns?


In [ ]:

var dns = require('dns');
var util = require('util');

function createZone(client, project, subdomain) {
    return client.request({
            url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones`,
            params: {
                dnsName: subdomain
            }
        })
        .then(res => {
            if(res.data.length > 0) {
                console.log(`zone ${subdomain} already mapped`);
                return Promise.resolve(res.data[0].name);
            }
            return client.request({
                method: 'POST',
                url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones`,
                data: {
                    name: 'mz-' + uuid().substr(0, 5) + safeName(subdomain),
                    dnsName: subdomain
                }
            });
        })
}

function createIP(client, project, subdomain) {
    var zone;
    return createZone(client, project, subdomain)
        .then(z => {
            zone = z;
            console.log(zone);
            return client.request({
                method: 'POST',
                url: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses`,
                data: {
                    name: 'ip-' + uuid().substr(0, 5) + safeName(subdomain)
                }
            });
        })
        .then(res => res.data.name)
        // TODO: update zone with A record
}

function addIP(project, subdomain) {
    var client, addresses;
    return util.promisify(dns.lookup)(subdomain)
        .then((addrs, family) => {
            console.log('addresses:', addresses);
            addresses = addrs;
        })
        .catch(up => {
            if(up.message.includes('ENOTFOUND')) {
                addresses = []
                return;
            }
            throw up
        })
        .then(() => authorizeGoogle(credentials))
        .then(client => (client = client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses`,
            params
        })))
        .then(res => {
            var addrs = res.data;
            console.log(addrs);
            var matching = addressses.filter(a => addrs.indexOf(a) > -1);
            if(matching.length > 0) {
                console.log(`domain ${subdomain} already exists, ip address ${matching[0]}`)
                return Promise.resolve(matching[0].name);
            }
            // check if there is an ip unassigned
            return createIP(client, project, subdomain);
        });
}

module.exports = addIP;


test check dns?

In [ ]:
var assert = require('assert');
var importer = require('../Core');
var addIP = importer.import('check dns');
var project = 'spahaha-ea443';
var bucketName = 'sheet-to-web.sheet-to-web.com';

describe('adding an external ip', () => {
    it('should add an ip', () => {
        return addIP(project, bucketName)
            .then(ip => {
                console.log(ip);
                assert(ip.length > 0, 'should have added an ip');
            })
    })
    
});


how to create a bucket?


In [ ]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials = path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

function listBuckets(project, bucketName) {
    var params = {project: project};
    if(bucketName) {
        params.prefix = bucketName;
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/storage/v1/b`,
            params
        }))
        .then(response => response.data.items || [])
}

function createBucket(project, bucketName) {
    var params = {project: project};
    return listBuckets(project, bucketName)
        .then(buckets => {
            if(Object.keys(buckets).length > 0) {
                console.log(`bucket ${bucketName} already exists`);
                return Promise.resolve(buckets[0].name);
            }
            return authorizeGoogle(credentials)
                .then(client => client.request({
                    method: 'POST',
                    url: `https://www.googleapis.com/storage/v1/b`,
                    params,
                    data: {
                        name: bucketName,
                        location: "us",
                        storageClass: "multi_regional"
                    }
                }))
                .then(response => response.data.name)
        })
}

module.exports = createBucket;


test create bucket?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var createBucket = importer.import('create a bucket');
var project = 'spahaha-ea443';
var bucketName = 'sheet-to-web.sheet-to-web.com';

describe('adding a bucket to google storage', () => {
    it('should add a bucket', () => {
        return createBucket(project, bucketName)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a bucket');
            })
    })
    
});
    

how to authorize google services?

In [ ]:
var {auth} = require('google-auth-library');
var jwt_client, google_auth;
var headers = {};
var util = require('util');
var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/compute',
    'https://www.googleapis.com/auth/cloud-platform'
];

function authorizeGoogle(creds) {
    return auth.getClient({
        keyFile: creds,
        scopes: GOOGLE_AUTH_SCOPE
    });
}

module.exports = authorizeGoogle;


test list url map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets
} = importer.import('list google bucket url map');
var project = 'spahaha-ea443';
var map = 'web-map';
var proxy = 'web-map-target-proxy-5';

describe('listing subdomains from load balancer', () => {
    
    it('should list buckets maps', () => {
        return listUrlMaps(project)
            .then(items => {
                console.log(items[Object.keys(items)[0]].hostRules);
                console.log(items[Object.keys(items)[0]].pathMatchers);
                console.log(items[Object.keys(items)[0]].pathMatchers[0].pathRules);
                assert(Object.keys(items).length > 0, 'should have a url mapper');
                
                return listTargetProxies(
                    project,
                    map);
            })
            .then(proxies => {
                console.log(proxies);
                assert(proxies.length > 0, 'should have a proxy');
            
                return listGlobalForwards(
                    project,
                    proxy);
            })
            .then(rules => {
                console.log(rules);
                assert(Object.keys(rules).length > 0, 'should have a rule');
            
                return listBackendBuckets(project);
            })
            .then(buckets => {
                console.log(buckets);
                assert(Object.keys(buckets).length > 0, 'should have a bucket');
            });
    })
    
    
})
